# Modelling a finite fault

## Import libraries

In [1]:
from FME.interpolators.piecewiselinear_interpolator import PiecewiseLinearInterpolator as PLI
from FME.supports.tet_mesh import TetMesh
from FME.modelling.features.geological_feature import GeologicalFeatureInterpolator
from FME.modelling.features.faulted_geological_feature import FaultedGeologicalFeature
from FME.visualisation.model_visualisation import LavaVuModelViewer
from FME.modelling.structural_frame import StructuralFrameBuilder
from FME.modelling.fault.fault_segment import FaultSegment

# other useful libraries
import numpy as np


## Define model mesh
Using boundary points. In the future this will be handled by a model support builder, where either a tetmesh or structured grid will be built. You can specify the resolution of the model by the number of tetrahedrons, this defines the degrees of freedom of the system

In [2]:
boundary_points = np.zeros((2,3))
boundary_points[0,0] = -20
boundary_points[0,1] = -20
boundary_points[0,2] = -20
boundary_points[1,0] = 20
boundary_points[1,1] = 20
boundary_points[1,2] = 20
mesh = TetMesh()
mesh.setup_mesh(boundary_points, 
                n_tetra=50000,)

### Create a fault frame
The fault frame needs to be given an interpolator to use to build the different scalar fields. In this case we link a PLI object to the model mesh.

In [6]:
fault_frame_interpolator = PLI(mesh)
fault_frame_builder = StructuralFrameBuilder(interpolator=fault_frame_interpolator,
                                             mesh=mesh,
                               name='FaultSegment1')

### Create and add data to the fault frame
The fault has a simple geometry and is dipping 35 degrees west.

In [7]:
for y in range(-15,15):
    fault_frame_builder.add_point([18.,y,18],0,itype='gy')
    fault_frame_builder.add_point([10,y,-5],1.,itype='gy')
    fault_frame_builder.add_strike_dip_and_value([18.,y,18],strike=180,dip=35,val=0,itype='gx')

The fault frame can be built by calling `build()` this can be given specific arguments to pass to the interpolator class. For example different weights or choice of solver. This will take some time to compute depending on the mesh resolution

In [8]:
fault_frame = fault_frame_builder.build()
fault = FaultSegment(fault_frame, displacement=4)

Building gx
Building gy
Creating analytical gz


### Building a stratigraphic horizon
Just build a flat lying stratigraphic horizon using two value control points and a strike and dip constraint. These can also be modified.

In [ ]:
interpolator = PLI(mesh)
feature_builder = GeologicalFeatureInterpolator(interpolator, name='stratigraphy')
feature_builder.add_point([0,0,0],0)
feature_builder.add_point([0,0,1],-0.5)
feature_builder.add_strike_and_dip([0,0,0],90,0)
solver = 'lu'
feature = feature_builder.build(solver=solver)

In [ ]:
faulted_feature = FaultedGeologicalFeature(feature, fault)

In [ ]:
viewer = LavaVuModelViewer()
viewer.plot_isosurface(faulted_feature,isovalue=0)
mask = fault_frame.features[0].support.get_node_values() > 0
mask[mesh.elements] = np.any(mask[mesh.elements] == True, axis=1)[:, None]
viewer.plot_points(mesh.nodes[mask], "nodes", col="red")
viewer.plot_isosurface(fault_frame.features[0], isovalue=0, colour='blue')
viewer.interactive()